In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

In [2]:
is_data_loaded = True
#is_data_loaded = False



In [125]:
fold = "5"

train_src_name = 'train_fold_'+fold
test_src_name = 'test_fold_'+fold
print("Using local data")
train_data = pd.read_csv('./'+train_src_name+'.csv')



test_data = pd.read_csv('./'+test_src_name+'.csv')


alt_train_data = pd.read_csv('./'+train_src_name+'.csv')


Using local data


In [126]:
len(train_data)

800

In [127]:
len(test_data)

200

## Preprocess Train Data and Finally split to valid

> Add blockquote



In [128]:
# Drop missing values
train_data.dropna(inplace=True)





In [129]:
label_col = 'Label'
cf_label_val = 'bad'
org_label_val = 'good'

print("Label Column: "+label_col)
# Separate features and labels
X_train_data = train_data.drop(label_col, axis=1)
y_train_data = train_data[label_col].apply(lambda x: 1 if x.strip() == cf_label_val else 0)  # Convert labels to binary

# Define preprocessing for numerical and categorical columns
numerical_features = X_train_data.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train_data.select_dtypes(include=['object']).columns

Label Column: Label


In [130]:
# Preprocessor (fit only on training data)
preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), numerical_features),
          ('cat', OneHotEncoder(drop='first',handle_unknown='ignore'), categorical_features)])

In [131]:
# Apply transformations
X_train_data = preprocessor.fit_transform(X_train_data)

# Convert sparse matrix to dense if necessary
if hasattr(X_train_data, 'toarray'):  # Check if X is sparse
    X_train_data = X_train_data.toarray()



In [132]:
len(y_train_data)

800

In [133]:
# Split data into train and valid sets
X_train, X_valid, y_train, y_valid = train_test_split(X_train_data, y_train_data, test_size=0.25, random_state=13)

In [134]:
print(len(y_train))
print(len(y_valid))

600
200


## Preprocess Test Data

In [135]:
# Now apply the same transformations to the test set
test_data.dropna(inplace=True)

# Separate features and labels for the test set
X_test_data = test_data.drop(label_col, axis=1)
y_test_data = test_data[label_col].apply(lambda x: 1 if x.strip() == cf_label_val else 0)  # Convert labels to binary

# Transform the test data using the already fitted preprocessor
X_test_data = preprocessor.transform(X_test_data)

# Convert sparse matrix to dense if necessary for the test set
if hasattr(X_test_data, 'toarray'):
    X_test_data = X_test_data.toarray()

# Now you can use X_test and y_test in your model evaluation or testing
print("Test set transformed successfully!")

X_test = X_test_data
y_test = y_test_data

Test set transformed successfully!


# Preprocess Alt Train Data

In [136]:
# Now apply the same transformations to the test set
alt_train_data.dropna(inplace=True)

# Separate features and labels for the test set
X_alt_train_data = alt_train_data.drop(label_col, axis=1)
y_alt_train_data = alt_train_data[label_col].apply(lambda x: 1 if x.strip() == '>50K' else 0)  # Convert labels to binary

# Transform the test data using the already fitted preprocessor
X_alt_train_data = preprocessor.transform(X_alt_train_data)

# Convert sparse matrix to dense if necessary for the test set
if hasattr(X_alt_train_data, 'toarray'):
    X_alt_train_data = X_alt_train_data.toarray()

# Now you can use X_test and y_test in your model evaluation or testing
print("Alt_Train set transformed successfully!")

X_alt_train = X_alt_train_data
y_alt_train = y_alt_train_data

Alt_Train set transformed successfully!


In [137]:
# Convert data to PyTorch tensors
class AdultDataset(Dataset):
    def __init__(self, X, y):
      self.X = torch.tensor(X, dtype=torch.float32)
      self.y = torch.tensor(y.values, dtype=torch.float32)

    def __len__(self):
      return len(self.y)

    def __getitem__(self, idx):
      return self.X[idx], self.y[idx]

In [138]:
train_dataset = AdultDataset(X_train, y_train)
test_dataset = AdultDataset(X_test, y_test)
valid_dataset = AdultDataset(X_valid, y_valid)

alt_train_dataset = AdultDataset(X_alt_train, y_alt_train)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

alt_train_loader =  DataLoader(alt_train_dataset, batch_size=32, shuffle=False)

In [139]:
# Define the optimized neural network
class BestTabularNN_2(nn.Module):
  def __init__(self, input_size):
    super(BestTabularNN_2, self).__init__()
    self.fc1 = nn.Linear(input_size, 31)   # First fully connected layer
    self.bn1 = nn.BatchNorm1d(31)          # Batch normalization
    self.dropout1 = nn.Dropout(0.3)         # Dropout layer

    self.fc2 = nn.Linear(31,19)          # Second fully connected layer
    self.bn2 = nn.BatchNorm1d(19)          # Batch normalization
    self.dropout2 = nn.Dropout(0.3)         # Dropout layer

    self.fc3 = nn.Linear(19,1)          # Second fully connected layer
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    x = torch.relu(self.bn1(self.fc1(x)))
    x = self.dropout1(x)
    x = torch.relu(self.bn2(self.fc2(x)))
    x = self.dropout2(x)
    x = self.sigmoid(self.fc3(x))

    return x



In [140]:
# Initialize the model, loss function, and optimizer
input_size = X_train.shape[1]
model = BestTabularNN_2(input_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)

In [141]:
'''
# Training loop
def train(model, train_loader, criterion, optimizer, epochs=20):
  model.train()
  for epoch in range(epochs):
    total_loss = 0
    for X_batch, y_batch in train_loader:
      optimizer.zero_grad()
      outputs = model(X_batch)
      loss = criterion(outputs.view(-1), y_batch)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}')
'''


"\n# Training loop\ndef train(model, train_loader, criterion, optimizer, epochs=20):\n  model.train()\n  for epoch in range(epochs):\n    total_loss = 0\n    for X_batch, y_batch in train_loader:\n      optimizer.zero_grad()\n      outputs = model(X_batch)\n      loss = criterion(outputs.view(-1), y_batch)\n      loss.backward()\n      optimizer.step()\n      total_loss += loss.item()\n    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}')\n"

In [142]:

# Function to train the model
def train(model, train_loader, criterion, optimizer, epochs=20, start_epoch=0):
  model.train()
  for epoch in range(start_epoch, start_epoch + epochs):
    total_loss = 0
    for X_batch, y_batch in train_loader:
      optimizer.zero_grad()
      outputs = model(X_batch)
      loss = criterion(outputs.view(-1), y_batch)
      loss.backward()
      optimizer.step()
      total_loss += loss.item()
    print(f'Epoch {epoch+1}/{start_epoch + epochs}, Loss: {total_loss/len(train_loader):.4f}')
    evaluate(model, valid_loader)
    save_model(model, optimizer, epoch, path='_model.pth')


# Save the model state
def save_model(model, optimizer, epoch, path='best_model.pth'):
  torch.save({
  'epoch': epoch,
  'model_state_dict': model.state_dict(),
  'optimizer_state_dict': optimizer.state_dict(),
  }, "epoch"+str(epoch+1)+path)


def load_model(model, optimizer, path='epoch5_model.pth'):
  checkpoint = torch.load(path)

  # Load model state dict
  model.load_state_dict(checkpoint['model_state_dict'])

  # Load optimizer state dict
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

  # Retrieve the saved epoch
  start_epoch = checkpoint['epoch']

  print(f"Model loaded from {path}, starting at epoch {start_epoch + 1}")
  return model, optimizer, start_epoch

In [143]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Evaluation loop
def evaluate(model, test_loader):
  model.eval()
  y_true = []
  y_pred = []

  with torch.no_grad():
    for X_batch, y_batch in test_loader:
      outputs = model(X_batch)
      predicted = (outputs.view(-1) > 0.5).float()

      # Append true labels and predicted labels
      y_true.extend(y_batch.numpy())
      y_pred.extend(predicted.numpy())

  # Calculate accuracy, precision, recall, and F1 score
  accuracy = accuracy_score(y_true, y_pred)
  precision = precision_score(y_true, y_pred)
  recall = recall_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)

  print(f'Accuracy: {accuracy:.4f}')
  print(f'Precision: {precision:.4f}')
  print(f'Recall: {recall:.4f}')
  print(f'F1 Score: {f1:.4f}')
  print("\nClassification Report:\n", classification_report(y_true, y_pred))




In [144]:
val_epochs = 50

In [145]:
# Train and save the model
#train(model, train_loader, criterion, optimizer, epochs=val_epochs)


In [146]:
'''
FOLD 1: 38/47/50
model, optimizer, start_epoch = load_model(model, optimizer, path='epoch20_model.pth')

FOLD 2
model, optimizer, start_epoch = load_model(model, optimizer, path='epoch19_model.pth')

FOLD 3
model, optimizer, start_epoch = load_model(model, optimizer, path='epoch20_model.pth')

FOLD 4
model, optimizer, start_epoch = load_model(model, optimizer, path='epoch3_model.pth')

FOLD 5
model, optimizer, start_epoch = load_model(model, optimizer, path='epoch5_model.pth')
'''

"\nFOLD 1: 38/47/50\nmodel, optimizer, start_epoch = load_model(model, optimizer, path='epoch20_model.pth')\n\nFOLD 2\nmodel, optimizer, start_epoch = load_model(model, optimizer, path='epoch19_model.pth')\n\nFOLD 3\nmodel, optimizer, start_epoch = load_model(model, optimizer, path='epoch20_model.pth')\n\nFOLD 4\nmodel, optimizer, start_epoch = load_model(model, optimizer, path='epoch3_model.pth')\n\nFOLD 5\nmodel, optimizer, start_epoch = load_model(model, optimizer, path='epoch5_model.pth')\n"

In [151]:
# Later, load the model and optimizer state, and resume training

#model, optimizer, start_epoch = load_model(model, optimizer, path='epoch86_model.pth')
model, optimizer, start_epoch = load_model(model, optimizer, path='m5_epoch8_model.pth')# 12

Model loaded from m5_epoch8_model.pth, starting at epoch 8


<ipython-input-142-a5ccf463de64>:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(path)


### Testing vs val loader to verify

In [152]:
evaluate(model, valid_loader)

Accuracy: 0.7600
Precision: 0.6136
Recall: 0.4655
F1 Score: 0.5294

Classification Report:
               precision    recall  f1-score   support

         0.0       0.80      0.88      0.84       142
         1.0       0.61      0.47      0.53        58

    accuracy                           0.76       200
   macro avg       0.71      0.67      0.68       200
weighted avg       0.75      0.76      0.75       200



In [153]:
evaluate(model, test_loader)

Accuracy: 0.7500
Precision: 0.6190
Recall: 0.4333
F1 Score: 0.5098

Classification Report:
               precision    recall  f1-score   support

         0.0       0.78      0.89      0.83       140
         1.0       0.62      0.43      0.51        60

    accuracy                           0.75       200
   macro avg       0.70      0.66      0.67       200
weighted avg       0.74      0.75      0.74       200



# Save into a new dataframe

In [154]:
# Evaluation function to add predictions to the original train data and save
def evaluate_and_save(model, train_loader, original_train_data, output_file='train_predictions.csv'):
    model.eval()
    y_pred = []

    # Collect predictions
    with torch.no_grad():
        for X_batch, _ in train_loader:  # We do not need true labels during prediction
            outputs = model(X_batch)
            predicted = (outputs.view(-1) > 0.5).float()  # Threshold at 0.5 for binary classification
            y_pred.extend(predicted.cpu().numpy())  # Detach and move to NumPy

    # Convert binary predictions to the original format ('>50K' and '<=50K')
    y_pred_str = [cf_label_val if y == 1 else org_label_val for y in y_pred]

    # Add predictions to the original train data
    original_train_data['Predicted_Label'] = y_pred_str

    # Save the DataFrame to a CSV file
    original_train_data.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}")



In [155]:
# Example usage:
# original_test_data is the DataFrame containing the original test data (with features, but without labels)
# test_loader is the DataLoader for the test set
#evaluate_and_save(model, alt_train_loader, alt_train_data, output_file='pred_'+train_src_name+'.csv')

In [156]:
evaluate_and_save(model, test_loader, test_data, output_file='pred_'+test_src_name+'.csv')

Predictions saved to pred_test_fold_5.csv
